In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [ ]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [93]:
df = fetch_hourly_rides(12)

2025-03-03 01:47:34,064 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:47:34,071 INFO: Initializing external client
2025-03-03 01:47:34,073 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:47:34,760 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649


AttributeError: 'Feature' object has no attribute 'dt'

In [ ]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-03 00:00:00+00:00,137,14
1,2025-03-03 00:00:00+00:00,131,0
2,2025-03-02 20:00:00+00:00,22,0
3,2025-03-02 18:00:00+00:00,46,0
4,2025-03-03 00:00:00+00:00,134,0
...,...,...,...
4011,2025-03-03 03:00:00+00:00,191,0
4012,2025-03-03 04:00:00+00:00,64,0
4013,2025-03-03 01:00:00+00:00,257,0
4014,2025-03-03 01:00:00+00:00,142,5


In [ ]:
df_pred = fetch_predictions(240)

2025-03-03 01:43:54,403 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:43:54,409 INFO: Initializing external client
2025-03-03 01:43:54,409 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:43:55,061 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.44s) 


In [ ]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,95,1.0,2025-03-03 03:00:00+00:00
1,179,1.0,2025-03-03 03:00:00+00:00
2,127,0.0,2025-03-03 03:00:00+00:00
3,90,71.0,2025-03-03 03:00:00+00:00
4,79,98.0,2025-03-03 03:00:00+00:00
...,...,...,...
748,93,1.0,2025-03-03 07:00:00+00:00
749,194,0.0,2025-03-03 07:00:00+00:00
750,36,0.0,2025-03-03 07:00:00+00:00
751,210,0.0,2025-03-03 07:00:00+00:00


In [ ]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [ ]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-03 03:00:00+00:00,50,1,21.0
1,2025-03-03 03:00:00+00:00,47,0,0.0
2,2025-03-03 03:00:00+00:00,32,0,0.0
3,2025-03-03 03:00:00+00:00,139,0,0.0
4,2025-03-03 03:00:00+00:00,3,0,0.0
...,...,...,...,...
246,2025-03-03 03:00:00+00:00,37,0,1.0
247,2025-03-03 03:00:00+00:00,40,0,0.0
248,2025-03-03 03:00:00+00:00,114,3,37.0
249,2025-03-03 03:00:00+00:00,14,0,1.0


In [ ]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [ ]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
141,2025-03-03 03:00:00+00:00,2,0,0.0,0.0
4,2025-03-03 03:00:00+00:00,3,0,0.0,0.0
185,2025-03-03 03:00:00+00:00,4,2,1.0,-1.0
191,2025-03-03 03:00:00+00:00,6,0,0.0,0.0
121,2025-03-03 03:00:00+00:00,7,1,2.0,1.0
...,...,...,...,...,...
39,2025-03-03 03:00:00+00:00,259,0,0.0,0.0
14,2025-03-03 03:00:00+00:00,260,0,1.0,1.0
171,2025-03-03 03:00:00+00:00,261,0,17.0,17.0
90,2025-03-03 03:00:00+00:00,262,2,89.0,87.0


In [ ]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-03 03:00:00+00:00,50,1,21.0,20.0
1,2025-03-03 03:00:00+00:00,47,0,0.0,0.0
2,2025-03-03 03:00:00+00:00,32,0,0.0,0.0
3,2025-03-03 03:00:00+00:00,139,0,0.0,0.0
4,2025-03-03 03:00:00+00:00,3,0,0.0,0.0
...,...,...,...,...,...
246,2025-03-03 03:00:00+00:00,37,0,1.0,1.0
247,2025-03-03 03:00:00+00:00,40,0,0.0,0.0
248,2025-03-03 03:00:00+00:00,114,3,37.0,34.0
249,2025-03-03 03:00:00+00:00,14,0,1.0,1.0


In [ ]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [ ]:
mae_by_hour["MAE"].mean()

14.589641434262948